In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [29]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [3]:
datasets = ['(3,4)']
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [6]:
context_unaware = False # whether original or context_unaware simulations are evaluated

setting = 'standard'
granularity_list = ['coarse', 'mixed', 'fine']
non_default_gran_list = ['coarse', 'fine']

### Determine vocab size

In [ ]:
import os
import csv
from collections import Counter

# Define the directory and filename for the CSV file
output_folder = 'analysis'
csv_filename = os.path.join(output_folder, '00_vocab_sizes.csv')

# Check if the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Check if the CSV file exists, if not, create it with headers
fieldnames = ['dataset', 'context_condition', 'vocab_size', 'symbols']
file_exists = os.path.exists(csv_filename)
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()

# Open the file in append mode and write the vocab sizes for each dataset
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Go through all datasets
    for i, d in enumerate(datasets):
        for g in granularity_list:
            # select first run
            if g != 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + 'granularity_' + g + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)

            elif g == 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)
            
            writer.writerow({'dataset': d, 'context_condition': g, 'vocab_size': vocab_size, 'symbols': symbol_counts})


In [32]:
import pandas as pd
df = pd.read_csv('analysis/00_vocab_sizes.csv')
df

,dataset,context_condition,vocab_size,symbols
0,"(3,4)",coarse,12,"Counter({0: 742, 14: 450, 9: 391, 4: 387, 6: 2..."
1,"(3,4)",mixed,15,"Counter({0: 1807, 6: 779, 2: 582, 8: 537, 11: ..."
2,"(3,4)",fine,14,"Counter({0: 742, 14: 332, 5: 315, 4: 214, 12: ..."
3,"(3,8)",coarse,17,"Counter({0: 4364, 16: 1585, 6: 1552, 17: 1485,..."
4,"(3,8)",mixed,28,"Counter({0: 11662, 25: 4555, 2: 2113, 12: 2050..."
5,"(3,8)",fine,27,"Counter({0: 4364, 23: 1292, 16: 947, 4: 872, 2..."
6,"(3,16)",coarse,32,"Counter({0: 29467, 18: 15057, 47: 10339, 41: 6..."
7,"(3,16)",mixed,52,"Counter({0: 83302, 47: 18222, 12: 13188, 37: 1..."
8,"(3,16)",fine,52,"Counter({0: 29467, 5: 8383, 22: 5202, 19: 4557..."
9,"(4,4)",coarse,15,"Counter({0: 3743, 4: 2910, 14: 2438, 15: 1895,..."


### See which messages are produced for which concepts

In [4]:
datasets = ['(3,4)',]
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [ ]:
# go through all datasets
for i, d in enumerate(datasets):
    for g in non_default_gran_list:
        if g != 'mixed':
        # select first run
            path_to_run = paths[i] + '/' + str(setting) +'/granularity_' + g +'/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'
            
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction.sender_input
        print(sender_input.shape)
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))

        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        #print(context_conds)

        # get random qualitative samples
        #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
        n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
        #n_fixed = 3
        fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
        #fixed_indices = [0, 2, 1]
        #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
        fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
        #fixed_values = [0, 1, 2]
        print(n_fixed, fixed_indices, fixed_values)
        #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
        # TODO: adapt this loop such that multiple indices can be fixed
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        #print(all_for_this_concept)                
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            df = pd.DataFrame(sample, columns=column_names)
            df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_'+g +'_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
            print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) +'_' + g + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + g + " could not be generated")

In [29]:
import pandas as pd
df = pd.read_csv('analysis/quali_(3,4)_standard_[0. 3. 2.],[1. 1. 1.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,1384,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 1, 0]"
1,90,[0. 3. 2.],[1. 1. 1.],1,"[1, 1, 1, 0]"
2,1384,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 1, 0]"
3,1183,[0. 3. 2.],[1. 1. 1.],1,"[11, 1, 1, 0]"
4,1798,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 9, 0]"
5,1483,[0. 3. 2.],[1. 1. 1.],2,"[1, 1, 14, 0]"
6,389,[0. 3. 2.],[1. 1. 1.],0,"[1, 1, 14, 0]"
7,1535,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 1, 0]"
8,1535,[0. 3. 2.],[1. 1. 1.],0,"[11, 1, 1, 0]"
9,1699,[0. 3. 2.],[1. 1. 1.],1,"[1, 1, 9, 0]"


### Count messages used for each dataset

In [ ]:
import csv
import os
import torch

# Define the directory and filename for the CSV file
analysis_folder = 'analysis'
csv_filename = os.path.join(analysis_folder, 'message_counts.csv')

fieldnames = ['dataset', 'context_condition', 'unique_messages']

# Check if the CSV file exists, if not, create it with headers
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Go through all datasets
for i, d in enumerate(datasets):
    for g in granularity_list:
        if g != 'mixed':
      # select first run
            path_to_run = paths[i] + '/' + str(setting) + '/granularity_' + g + '/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'

        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        unique_messages = set(tuple(msg) for msg in messages)  # Convert lists to tuples before adding to the set

        mess_count = len(unique_messages)
        # Check if the combination of dataset and context condition already exists in the CSV file
        combination_exists = False
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row['dataset'] == d and row['context_condition'] == g:
                    combination_exists = True
                    break

        # If the combination exists, update the corresponding row
        if combination_exists:
            rows = []
            with open(csv_filename, 'r', newline='') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    if row['dataset'] == d and row['context_condition'] == g:
                        row['unique_messages'] = mess_count
                    rows.append(row)
            
            with open(csv_filename, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
        # If the combination does not exist, append a new row
        else:
            with open(csv_filename, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'dataset': d, 'context_condition': g, 'unique_messages': mess_count})


In [35]:
import pandas as pd
df = pd.read_csv('analysis/message_counts.csv')
df

,dataset,context_condition,unique_messages
0,"(3,4)",coarse,33
1,"(3,4)",mixed,175
2,"(3,4)",fine,133
3,"(3,8)",coarse,41
4,"(3,8)",mixed,803
5,"(3,8)",fine,438
6,"(3,16)",coarse,133
7,"(3,16)",mixed,4032
8,"(3,16)",fine,1308
9,"(4,4)",coarse,204


## Concepts and messages

In [36]:
import sklearn

### D(3,4)